In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions
import spacy
import nltk
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from nltk.corpus import wordnet

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Ensure Spacy model is installed
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    from spacy.cli import download
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

import pprint
# Set the display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# Load the CSV file
file_path = 'output.csv'
dataset_name = pd.read_csv(file_path)

dataset_name.fillna('', inplace=True)
dataset_name = dataset_name.astype(str)  # Cast the entire DataFrame to object type

C:\Users\SimWen\AppData\Local\Temp\ipykernel_27188\1503495290.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset_name.fillna('', inplace=True)


In [6]:
df_input = pd.DataFrame()

for i in range(len(dataset_name)):

    question_txt = dataset_name.loc[i,'3. Failure cause (FC) of the work element']
    question = "Failure cause of the work element : \n" + question_txt + "\n\n"
    
    failure_mode_txt = dataset_name.loc[i,'2. Failure mode (FM) of the focus element']
    failure_mode = "Failure mode of the focus element : \n" + failure_mode_txt + "\n\n"

    failure_effect_txt = dataset_name.loc[i,'1. Failure effect (FE) to next higher level element and/or end user']
    failure_effect = "Failure effect to next higher level element : \n" + failure_effect_txt + "\n\n"

    current_preventive_control_txt = dataset_name.loc[i,'Current preventive control (PC) for FC']
    current_preventive_control = "Current preventive control : \n" + current_preventive_control_txt + "\n\n"

    current_detection_control_txt = dataset_name.loc[i,'Current detection control (DC) for FC or FM']
    current_detection_control = "Current detection control : \n" + current_detection_control_txt + "\n\n"

    recommended_action_txt = dataset_name.loc[i,'Recommended action']
    recommended_action = "Recommended_action : \n" + recommended_action_txt + "\n\n"

    function_of_process_work_txt = dataset_name.loc[i,'2. Function of the process step and product characteristic']
    function_of_process_work = "Function of the process step : \n" + function_of_process_work_txt + "\n\n"

    function_of_process_step_txt = dataset_name.loc[i,'3. Function of the process work element and process characteristic']
    function_of_process_step = "Function of the process work element : \n" + function_of_process_step_txt + "\n\n"

    location_txt = dataset_name.loc[i,'location']
    sme_remark = "Manufacturing Location : \n" + location_txt + "\n\n"

    sme_remark_txt = dataset_name.loc[i,'SME_Remark']
    sme_remark = "SME Remark : \n" + sme_remark_txt + "\n\n"
    

    # category_txt = dataset_name.loc[i,'5M']
    # category = "5M : \n" + category_txt + "\n\n"
    
    FMEA_txt = dataset_name.loc[i,'fmea_type']
    Name_of_the_unit_process_txt = dataset_name.loc[i,'subject']
    Package_code_txt = dataset_name.loc[i,'package']
    Document_Nr_txt = dataset_name.loc[i,'fmea_id']
    FMEA_idx_id_txt = dataset_name.loc[i,'fmea_idx_id']
    others_info = "Others Info : " + "\n" + "FMEA Type : " +  FMEA_txt +  "\n" + "Name of the unit process : " + Name_of_the_unit_process_txt + "\n" + "Package code : " + Package_code_txt + '\n' + "Manufacturing Location : " + location_txt #+"\n" + "FMEA ID : " + Document_Nr_txt + "\n" + "FMEA idx ID : " + FMEA_idx_id_txt                            
    

    
    
    # Create a list of dictionaries with the data
    data = [
        {
            'context': sme_remark + question +  failure_mode + failure_effect + current_preventive_control + current_detection_control +
                       recommended_action + function_of_process_work + function_of_process_step + others_info,
            'question': question,

        }
    ]

    df_data = pd.DataFrame(data)
    df_input = pd.concat([df_input, df_data], axis = 0)


df_input.reset_index(inplace=True)

dataset_name_processed = pd.concat([df_input, dataset_name], axis = 1)
dataset_name_processed.drop('index', axis=1, inplace=True)

dataset_name_processed.head(1)

,context,question,"1. Function of the process item - function of the system, sub-system, part element or process",Specification of process item,2. Function of the process step and product characteristic,No.,Requirement / Specification of the process step,Filter code,3. Function of the process work element and process characteristic,Specification of the process work element,1. Failure effect (FE) to next higher level element and/or end user,Severity (S) of FE,Severity (S max) of FE,2. Failure mode (FM) of the focus element,3. Failure cause (FC) of the work element,Current preventive control (PC) for FC,Occurrence (O1) of FC,Current detection control (DC) for FC or FM,Detection (D1) of FC/FM,PFMEA AP1,Action bundle (date),Recommended action,P/D,Name of responsible person,Target completion date,Status,Action taken with pointer to evidence,P/D.1,Completion date,Severity (S),Occurrence (O2),Detection (D2),PFMEA AP2,subject,fmea_id,fmea_type,package,location,SME_Remark,Index,fmea_idx_id
0,SME Remark : \n\n\nFailure cause of the work element : \nWrong wafer barcode\n[Material_Z8K00018355]\n\nFailure mode of the focus element : \nMixed Device\n\nFailure effect to next higher level element : \nCustomer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]\n\nCurrent preventive control : \n(P / D) Machine barcode scanner.\n\nCurrent detection control : \n(D) Machine wafer mapping reference coordinate detection stop machine for futher processing\n\nRecommended_action : \n\n\nFunction of the process step : \n00 Receiving - Incoming from previous process (Storage/Handling/ Transportation)\n\nFunction of the process work element : \nMH receive wafer from Pre-assembly through pass box to DA kitting storage rack using trolley (OJTI-Z8K00010746) MH will receive leadframe from mini store through pass box to DA kitting storage rack using trolley (OJTI-Z8K00005094) ACR personnel will receive glue through transit room to ACR room using trolley (OJTI-Z9K00010452)\n[Material_Z8K00018355]\n\nOthers Info : \nFMEA Type : Process\nName of the unit process : Unit Process FMEA of QFP+ Die attach & DA curing\nPackage code : All QFP package\nManufacturing Location : Manufacturing-Melaka,Failure cause of the work element : \nWrong wafer barcode\n[Material_Z8K00018355]\n\n,Function for Effects\n[Z8K00018355],,00 Receiving - Incoming from previous process (Storage/Handling/ Transportation),1.1.a,,,MH receive wafer from Pre-assembly through pass box to DA kitting storage rack using trolley (OJTI-Z8K00010746) MH will receive leadframe from mini store through pass box to DA kitting storage rack using trolley (OJTI-Z8K00005094) ACR personnel will receive glue through transit room to ACR room using trolley (OJTI-Z9K00010452)\n[Material_Z8K00018355],,Customer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355],8.0\n7.0,8.0,Mixed Device,Wrong wafer barcode\n[Material_Z8K00018355],(P / D) Machine barcode scanner.,3.0,(D) Machine wafer mapping reference coordinate detection stop machine for futher processing,2.0,L,,,,,,,,,,,,,,Unit Process FMEA of QFP+ Die attach & DA curing,Z8K00018355,Process,All QFP package,Manufacturing-Melaka,,0,Z8K00018355_0


In [7]:
df = dataset_name_processed
# Check if all values in a column are blank (NaN or empty string)
blank_cols = [col for col in df.columns if df[col].isnull().all() or (df[col] == '').all()]
df.head().T

,0,1,2,3,4
context,SME Remark : \n\n\nFailure cause of the work element : \nWrong wafer barcode\n[Material_Z8K00018355]\n\nFailure mode of the focus element : \nMixed Device\n\nFailure effect to next higher level element : \nCustomer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]\n\nCurrent preventive control : \n(P / D) Machine barcode scanner.\n\nCurrent detection control : \n(D) Machine wafer mapping reference coordinate detection stop machine for futher processing\n\nRecommended_action : \n\n\nFunction of the process step : \n00 Receiving - Incoming from previous process (Storage/Handling/ Transportation)\n\nFunction of the process work element : \nMH receive wafer from Pre-assembly through pass box to DA kitting storage rack using trolley (OJTI-Z8K00010746) MH will receive leadframe from mini store through pass box to DA kitting storage rack using trolley (OJTI-Z8K00005094) ACR personnel will receive glue through transit room to ACR room using trolley (OJTI-Z9K00010452)\n[Material_Z8K00018355]\n\nOthers Info : \nFMEA Type : Process\nName of the unit process : Unit Process FMEA of QFP+ Die attach & DA curing\nPackage code : All QFP package\nManufacturing Location : Manufacturing-Melaka,"SME Remark : \n\n\nFailure cause of the work element : \nMixed Leadframe\n[Method_Z8K00018355]\n\nFailure mode of the focus element : \nMixed Device\n\nFailure effect to next higher level element : \nCustomer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]\n\nCurrent preventive control : \n(P / D) LF barcode scanner\n\nCurrent detection control : \n(D) LF subpitch error detection stop machine, lot will not able to track in camstar\n\nRecommended_action : \n\n\nFunction of the process step : \n10 Material Preparation (Storage/Handling/ Transportation)\n\nFunction of the process work element : \nDA LA will standby lot (wafer and leadframe) at DA standby cabinet (OJTI-Z8K00010746) To get ready all necessary material, software, system prior to start production from planner/ supervisor such as production machine setup (bonding recipe, collet, collet holder, collet jig needle, needle holder, paperpot, glue, glue adaptor, adaptor lock stamp code, showerhead/writter) and leadframe. (OJTI-Z8K00004781)\n[Method_Z8K00018355]\n\nOthers Info : \nFMEA Type : Process\nName of the unit process : Unit Process FMEA of QFP+ Die attach & DA curing\nPackage code : All QFP package\nManufacturing Location : Manufacturing-Melaka","SME Remark : \n\n\nFailure cause of the work element : \nwrong glue used\n[Method_Z8K00018355]\n\nFailure mode of the focus element : \nLow die shear strength\n\nFailure effect to next higher level element : \nCustomer: Reliability failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]\n\nCurrent preventive control : \n(P / D) Glue barcode scanner\n\nCurrent detection control : \n(D) Camstar will prompt error when lot is track in and block lot from proceed in camstar.\n\nRecommended_action : \n\n\nFunction of the process step : \n10 Material Preparation (Storage/Handling/ Transportation)\n\nFunction of the process work element : \nDA LA will standby lot (wafer and leadframe) at DA standby cabinet (OJTI-Z8K00010746) To get ready all necessary material, software, system prior to start production from planner/ supervisor such as production machine setup (bonding recipe, collet, collet holder, collet jig needle, needle holder, paperpot, glue, glue adaptor, adaptor lock stamp code, showerhead/writter) and leadframe. (OJTI-Z8K00004781)\n[Method_Z8K00018355]\n\nOthers Info : \nFMEA Type : Process\nName of the unit process : Unit Process FMEA of QFP+ Die attach & DA curing\nPackage code : All QFP package\nManufacturing Location : Manufacturing-Melaka","SME Remark : \n\n\nFailure cause of the work element : \nwrong glue used\n[Method_Z8K00018355]\n\nFailure mode of the focus element : \nLow bond line thickness\n\nFailure effect to next higher level element : \nCustomer: Electrical

In [8]:
# Define a function to convert mixed case to lower case
def convert_mixed_case_to_lower(x):
    if isinstance(x, str):
        lines = x.split('\n')
        new_lines = []
        for line in lines:
            words = line.split()
            new_words = []
            for word in words:
                if word.isupper():
                    new_words.append(word)
                else:
                    new_words.append(word.lower())
            new_lines.append(' '.join(new_words))
        return ' '.join(new_lines)
    return x

# Function to remove special character or ID patterns
def remove_special_ids(text, pattern=r'\b\w*\d\w*\b'):
    if isinstance(text, str):
        return re.sub(pattern, ' ', text)
    return text

def remove_punctuation(text):
        text = re.sub(r'/', 'or', text)
        text = re.sub(r'&', 'and', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'(?<!\d)\.(?!\d)|[^\w\s.]', ' ', text)
        return text

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def stemming(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

def lemmatization(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def expand_contractions(text):
    return contractions.fix(text)

def remove_numbers(text):
    return re.sub(r'\d+', ' ', text)

def remove_extra_whitespace(text):
    return ' '.join(text.split())

# def remove_special_chars(df, columns):
#     for col in columns:
#         df[col] = df[col].apply(lambda x: re.sub(r'[^\w\s]', '', x))
#     return df

def preprocess(text):
    text = convert_mixed_case_to_lower(text)
    text = remove_special_ids(text)
    text = expand_contractions(text)
    text = remove_punctuation(text)
    # text = remove_numbers(text)
    # text = remove_special_chars(text)
    text = remove_extra_whitespace(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)  # or use stemming(tokens)
    return ' '.join(tokens)

In [9]:
import pandas as pd
import json
from dataclasses import dataclass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModel
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from haystack.document_store.faiss import FAISSDocumentStore
# from haystack.nodes import EmbeddingRetriever



c:\Users\SimWen\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:

@dataclass
class Document:
    metadata: dict
    page_content: str
    
documents = []
for index, row in df.iterrows():
    metadata = {
        'question': f'{row["question"]}',
        'Function of the process step and product characteristic' : f'{row["2. Function of the process step and product characteristic"]}',
        'Failure effect FE to next higher level element and/or end user': f'{row["1. Failure effect (FE) to next higher level element and/or end user"]}',
        'Failure mode FM of the focus element' : f'{row["2. Failure mode (FM) of the focus element"]}',
        'Current preventive control PC for FC' : f'{row["Current preventive control (PC) for FC"]}',
        'Current detection control DC for FC or FM' : f'{row["Current detection control (DC) for FC or FM"]}',
        'subject' : f'{row["subject"]}',
        'package' : f'{row["package"]}',
        'location' :f'{row["location"]}',
        'fmeaIid' :f'{row["fmea_id"]}',
        'fmeaidx_id' :f'{row["fmea_idx_id"]}'
    }
    page_content = f'"{row["context"]}"'
    page_content = preprocess(page_content)
    document = Document(metadata, page_content)
    documents.append(document)

In [11]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(documents)

# check the result after text spliting

data = []
for i, chunk in enumerate(docs):
    data.append({'Chunk ID': i, 'Text': chunk})

df = pd.DataFrame(data)
df.head().T

,0,1,2,3,4
Chunk ID,0,1,2,3,4
Text,"page_content='SME remark failure work element wrong wafer barcode failure mode focus element mixed device failure effect next higher level element customer electrical reliability failure IFX yield loss current preventive control P D machine barcode scanner current detection control D machine wafer mapping reference coordinate detection stop machine futher processing recommended_action function process step receiving incoming previous process storageorhandlingor transportation function process work element MH receive wafer pre assembly pas box DA kitting storage rack using trolley OJTI MH receive leadframe mini store pas box DA kitting storage rack using trolley OJTI ACR personnel receive glue transit room ACR room using trolley OJTI others info FMEA type process name unit process unit process FMEA QFP die attach DA curing package code QFP package manufacturing location manufacturing melaka' metadata={'question': 'Failure cause of the work element : \nWrong wafer barcode\n[Material_Z8K00018355]\n\n', 'Function of the process step and product characteristic': '00 Receiving - Incoming from previous process (Storage/Handling/ Transportation)', 'Failure effect FE to next higher level element and/or end user': 'Customer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]', 'Failure mode FM of the focus element': 'Mixed Device', 'Current preventive control PC for FC': '(P / D) Machine barcode scanner.', 'Current detection control DC for FC or FM': '(D) Machine wafer mapping reference coordinate detection stop machine for futher processing', 'subject': 'Unit Process FMEA of QFP+ Die attach & DA curing', 'package': 'All QFP package', 'location': 'Manufacturing-Melaka', 'fmeaIid': 'Z8K00018355', 'fmeaidx_id': 'Z8K00018355_0'}","page_content='SME remark failure work element mixed leadframe failure mode focus element mixed device failure effect next higher level element customer electrical reliability failure IFX yield loss current preventive control P D LF barcode scanner current detection control D LF subpitch error detection stop machine lot able track camstar recommended_action function process step material preparation storageorhandlingor transportation function process work element DA LA standby lot wafer leadframe DA standby cabinet OJTI get ready necessary material software system prior start production planneror supervisor production machine setup bonding recipe collet collet holder collet jig needle needle holder paperpot glue glue adaptor adaptor lock stamp code showerheadorwritter leadframe OJTI others info FMEA type process name unit process unit process FMEA QFP die attach DA curing package code QFP package manufacturing location manufacturing melaka' metadata={'question': 'Failure cause of the work element : \nMixed Leadframe\n[Method_Z8K00018355]\n\n', 'Function of the process step and product characteristic': '10 Material Preparation (Storage/Handling/ Transportation)', 'Failure effect FE to next higher level element and/or end user': 'Customer: Electrical / reliability Failure\n[Z8K00018355]\nIFX: Yield loss\n[Z8K00018355]', 'Failure mode FM of the focus element': 'Mixed Device', 'Current preventive control PC for FC': '(P / D) LF barcode scanner', 'Current detection control DC for FC or FM': '(D) LF subpitch error detection stop machine, lot will not able to track in camstar', 'subject': 'Unit Process FMEA of QFP+ Die attach & DA curing', 'package': 'All QFP package', 'location': 'Manufacturing-Melaka', 'fmeaIid': 'Z8K00018355', 'fmeaidx_id': 'Z8K00018355_1'}","page_content='SME remark failure work element wrong glue used failure mode focus element low die shear strength failure effect next higher level element customer reliability failure IFX yield loss current preventive control P D glue barcode scanner current detection control D camstar prompt error lot track block lot proceed camstar recommended_action function process step material preparation storag

In [12]:

# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [13]:
db = FAISS.from_documents(docs, embeddings)

In [14]:
# save the db (vector store ) to local
with open("multi-qa-MiniLM-L6-cos-v1.pkl", "wb") as f:
    pickle.dump(db, f)

In [15]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions
import spacy
import nltk
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from nltk.corpus import wordnet

import json
from dataclasses import dataclass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModel
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# load require package

import langchain
from langchain.document_loaders import HuggingFaceDatasetLoader
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, FalconForQuestionAnswering, FalconConfig
import transformers
import torch

from langchain.vectorstores import VectorStore
import pprint
import random
import string

from fuzzywuzzy import fuzz

c:\Users\SimWen\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [17]:
# load vectorstore.pkl file from local
vectorstore_file = "multi-qa-MiniLM-L6-cos-v1.pkl"

with open(vectorstore_file, "rb") as f:
    global vectorstore
    db: VectorStore = pickle.load(f)